In [1]:
import logging
import threading
import time

import numpy

import samlab.interactive
import samlab.timeseries

logging.basicConfig(level=logging.DEBUG)

In [2]:
stop = samlab.interactive.Stop()

def random_walk(key, count, mean, variance, period, writer):
    for i in range(count):
        if stop.triggered:
            break
        writer.add_scalar(key, i, numpy.random.normal(mean, variance))
        time.sleep(period)

with samlab.timeseries.Writer(experiment="Experiment 3") as writer:
    stop = samlab.interactive.Stop()
    
    threads = []
    threads.append(threading.Thread(target=random_walk, args=("foo/reward", 60, 0, 0.1, 0.5, writer)))
    threads.append(threading.Thread(target=random_walk, args=("bar/reward", 60, 0, 0.1, 0.5, writer)))

    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()

In [3]:
import samlab.database
db, fs = samlab.database.connect()